In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as pu

In [2]:
e = cptac.Endometrial()

In [17]:
prot_and_mutations = e.join_omics_to_mutations(
            mutations_genes = 'PTEN', omics_df_name = 'proteomics', omics_genes = 'PTEN', 
    mutations_filter = ['Nonsense_Mutation','Frame_Shift_Del','Frame_Shift_Ins'])
prot_and_mutations = prot_and_mutations.loc[prot_and_mutations['PTEN_Mutation'] !='Wildtype_Tumor']
prot_and_mutations = prot_and_mutations.loc[prot_and_mutations['PTEN_Mutation'] !='Wildtype_Normal']
prot_and_mutations.PTEN_Mutation.value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 69 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 3)


Nonsense_Mutation    25
Frame_Shift_Del      22
Missense_Mutation    21
Frame_Shift_Ins       6
Splice_Site           1
Name: PTEN_Mutation, dtype: int64

Format df for MutPlot (lollipop plot creation)

In [18]:
# drop unnecesary columns
d = ['PTEN_proteomics','PTEN_Mutation_Status','Sample_Status']
df = prot_and_mutations.drop(columns=d)
# Create Hugo_Symbol column
df['Hugo_Symbol'] = 'PTEN'
# Rename columns
df = df.reset_index()
df1 = df.rename(columns={'Patient_ID': 'Sample_ID', 'PTEN_Mutation': 'Mutation_Type',
                         'PTEN_Location':'Protein_Change'})
# Keep only Truncation type mutations
df2 = df1.loc[df1['Mutation_Type'] != 'Missense_Mutation']
df3 = df2.loc[df1['Mutation_Type'] != 'Splice_Site']
df3.Mutation_Type.value_counts()

Nonsense_Mutation    25
Frame_Shift_Del      22
Frame_Shift_Ins       6
Name: Mutation_Type, dtype: int64

In [22]:
fs = df3.loc[df1['Mutation_Type'] == 'Nonsense_Mutation']
fs.Protein_Change.value_counts()

p.R233*    4
p.R130*    4
p.E7*      3
p.E99*     2
p.W111*    1
p.Q245*    1
p.Y180*    1
p.Y27*     1
p.Q214*    1
p.Q171*    1
p.Q97*     1
p.C304*    1
p.S287*    1
p.G293*    1
p.G209*    1
p.W274*    1
Name: Protein_Change, dtype: int64

In [12]:
df3.to_csv('lollipop_endo.csv')

Extra

In [23]:
# Create lollipop plot with somatic mutations (counts all mutations - including multiple mutations in same sample)
mut = e.get_somatic_mutation()
pten = mut.loc[mut['Gene'] == 'PTEN']
df = pten.reset_index()
df1 = df.rename(columns={'Patient_ID': 'Sample_ID','Gene':'Hugo_Symbol',
                         'Mutation': 'Mutation_Type','Location':'Protein_Change'})
df2 = df1.loc[df1['Mutation_Type'] != 'Missense_Mutation']
df3 = df2.loc[df1['Mutation_Type'] != 'Splice_Site']
df3.Mutation_Type.value_counts()

Nonsense_Mutation    27
Frame_Shift_Del      26
Frame_Shift_Ins      11
Name: Mutation_Type, dtype: int64

In [24]:
l = df3.loc[df1['Mutation_Type'] == 'Frame_Shift_Ins']
l.Protein_Change.value_counts()

p.N323Kfs*2     2
p.P248Tfs*5     2
p.E242*         1
p.D268Gfs*30    1
p.M134Nfs*46    1
p.V166Sfs*14    1
p.H118Qfs*8     1
p.L295Sfs*3     1
p.I168Sfs*13    1
Name: Protein_Change, dtype: int64

In [21]:
df3.to_csv('lollipop_endo.csv')